In [1]:
import pandas as pd
import time
from multiprocessing import Pool
from sklearn.model_selection import ParameterGrid

In [8]:
def preProcessData(stockName, ST, LT):
    df = pd.read_csv("IntradayData_2018/"+stockName+".txt")
    df.columns = ['stock', 'date','time','openingPrice','high','low','closingPrice','volume','e']
    df = df.drop('e', 1)
    for i in ST:
        df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
    for i in LT:
        df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
    dateList = list(set(df.date.values))
    dateList = sorted(dateList, key=abs, reverse=False)
    return df, dateList

In [9]:
def convertTime(timeString):
    timeArr = timeString.split(':')
    time = int(timeArr[0])*60 + int(timeArr[1])
    return time

def revConvertTime(timeInt):
    timeHr = timeInt/60
    timeMin = timeInt%60
    return str(timeHr) + ':' + str(timeMin)

def sellIt(buyPrice, sellPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice - buyPrice*(1.0002))/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def buyIt(sellPrice, buyPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice - buyPrice*(1.0002))/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def parseRow(row, ST, LT):
    time = convertTime(row[1].time)
    openingPrice = row[1].openingPrice
    low = row[1].low
    high = row[1].high
    closingPrice = row[1].closingPrice
    listPrice = 0.5*(openingPrice + closingPrice)
    diff = row[1]["MA_"+str(LT)] - row[1]["MA_"+str(ST)]
    return time, openingPrice, low, high, closingPrice, listPrice, diff

In [10]:
def dayScript(df, ST, LT, money, targetPercentage, stopLossPercentage, entryDifference, verbose = False):
    eomDays = 0
    boughtFlag = 0
    dailyTrades = 0
    buyPrice = 0
    sellPrice = 0
    sellMargin = 1 + targetPercentage
    buyMargin = 1 - targetPercentage

#     stopLoss = 1 - stopLossPercentage
#     stopGain = 1 + stopLossPercentage

    for row in df.iterrows():
        time, openingPrice, low, high, closingPrice, listPrice, diff = parseRow(row, ST, LT)       
        if(time >= 570 and time <900):
            if(boughtFlag == 0):
                #market enter logic
                if(diff>0 and diff<entryDifference*listPrice):
                    #buy
                    buyPrice = listPrice
                    sellPrice = 0
                    timeBought = time
                    timeSold = 0
                    boughtFlag = 1
                    stopLossPrice = buyPrice * (1 - stopLossPercentage)
                    stopLossWindow = buyPrice*stopLossPercentage
                elif(diff<0 and diff>-1*entryDifference*listPrice):
                    #sell
                    sellPrice = listPrice
                    buyPrice = 0
                    timeSold = time
                    timeBought = 0
                    boughtFlag = -1
                    stopGainPrice = sellPrice * (1 + stopLossPercentage)
                    stopGainWindow = sellPrice*stopLossPercentage
            elif(boughtFlag==1):
                #market exit logic (when already bought)
                if(high > buyPrice*sellMargin):
                    percentageProfit = sellIt(buyPrice, buyPrice*sellMargin, timeBought, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
#                     print('target exit')
                elif(stopLossPrice > low):
                    percentageProfit = sellIt(buyPrice, stopLossPrice, timeBought, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
#                     print('stopLoss exit')
            else:
                #market exit logic (when already sold)
                if(sellPrice*buyMargin > low):
                    percentageProfit = buyIt(sellPrice, sellPrice*buyMargin, timeSold, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
#                     print('target exit')
                elif(high > stopGainPrice):
                    percentageProfit = buyIt(sellPrice, stopGainPrice, timeSold, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
#                     print('stopLoss exit')
            #trailing stopLoss stopGain calculations.
            if(boughtFlag ==1 ):
                trail = (listPrice - stopLossPrice)
                if(trail > stopLossWindow ):
                    climb = trail - stopLossWindow
                    stopLossPrice+= climb
                    
            elif(boughtFlag ==-1 ):
                trail = (stopGainPrice - listPrice)
                if(trail > stopGainWindow ):
                    climb = trail - stopGainWindow
                    stopGainPrice-= climb
                
        elif(time >= 900):
            if(boughtFlag == 1):
                percentageProfit =sellIt(buyPrice, closingPrice, timeBought, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1
            elif(boughtFlag == -1):
                percentageProfit = buyIt(sellPrice, closingPrice, timeSold, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1

#     print('Total Profit : ',totalProfit)
    return money, dailyTrades, eomDays

In [11]:
def movingAverageExperiment(paramList):
    df, dateList, ST,LT,targetPercentage,stopLossPercentage,entryDifference = paramList
    money = 1
    for date in dateList:
        new_df = df[(df.date == date)]
        updatedMoney, dailyTrades, eomDays = dayScript(new_df, ST, LT,  money, targetPercentage, stopLossPercentage, entryDifference, verbose = False)
#         print(date, money, updatedMoney, dailyTrades)
        money = updatedMoney

#     print('Eom days - ',eomDays)
    yearlyProfitPercentage = (money - 1) * 100
    print(ST,LT,targetPercentage,stopLossPercentage,entryDifference, 'Total Profit %= ', yearlyProfitPercentage)
   
    return yearlyProfitPercentage

In [12]:
# #old
# ST = [1,4,12,20]
# LT = [100,200,500,1000]
# targetPercentage = [0.01, 0.05]
# stopLossPercentage = [0.002, 0.006, 0.008, 0.010]
# entryDifference = [0.00001, 0.00005, 0.00010, 0.0002]
# # #new
# ST = [10,15,20,25]
# LT = [75,100,200,500]
# targetPercentage = [0.01, 0.05, 0.075]
# stopLossPercentage = [0.006, 0.007, 0.008, 0.010]
# entryDifference = [0.00005, 0.000075, 0.0001]
# # #v3
ST = [15,18,20]
LT = [100,120]
targetPercentage = [0.01, 0.05, 0.075]
stopLossPercentage = [0.006, 0.007, 0.008]
entryDifference = [0.00005, 0.000075, 0.0001]

paramList = [LT, ST, targetPercentage, stopLossPercentage, entryDifference]

In [13]:
# from sklearn.grid_search import ParameterGrid

param_grid = {'ST' : ST, 'LT': LT, 'targetPercentage': targetPercentage, 'stopLossPercentage': stopLossPercentage, 'entryDifference':entryDifference,}

grid = ParameterGrid(param_grid)

# ST_list = []
# LT_list = []
# targetPercentageList = []
# stopLossPercentageList = []
# entryDifferenceList = []
csvList = []
df, dateList = preProcessData('RELIANCE', ST, LT)

threadPoolSize = 24
pool = Pool(threadPoolSize)
resultList = {}
i=0
print('param combination = ' ,len(grid))
while i<len(grid):
    param_list =[]
    for j in range(threadPoolSize):
        if(i+j < len(grid)):
            params = grid[i+j]
            param_list.append([df, dateList, params['ST'], params['LT'],params['targetPercentage'], params['stopLossPercentage'],params['entryDifference']])
#         result = pool.apply_async(movingAverageExperiment, [df, dateList, params['ST'], params['LT'],params['targetPercentage'], params['stopLossPercentage'],params['entryDifference']])
#         params['result'] = result.get(timeout)
#         csvList.append(params)
    start = time.time()
    resultList = pool.map(movingAverageExperiment, param_list)
    end = time.time()
    for j in range(len(param_list)):
        params = param_list[j]
        result = resultList[j]
        newListVariable = {}
        newListVariable['params'] = params
        newListVariable['result']= result
        csvList.append(newListVariable)
    print('Time - ',end - start)
    i+=threadPoolSize
    print(i,len(grid))
    break

('param combination = ', 162)
(15, 100, 0.01, 0.006, 5e-05, 'Total Profit %= ', 23.213999465560665)
(15, 100, 0.05, 0.006, 5e-05, 'Total Profit %= ', 14.547974437768275)
(15, 100, 0.075, 0.006, 5e-05, 'Total Profit %= ', 14.547974437768275)
(15, 100, 0.01, 0.007, 5e-05, 'Total Profit %= ', 26.017420565978533)
(15, 100, 0.05, 0.007, 5e-05, 'Total Profit %= ', 17.821452868686084)
(15, 100, 0.05, 0.008, 5e-05, 'Total Profit %= ', 24.006317233830956)
(15, 100, 0.075, 0.007, 5e-05, 'Total Profit %= ', 17.821452868686084)
(15, 100, 0.01, 0.008, 5e-05, 'Total Profit %= ', 26.44986564643721)
(15, 100, 0.075, 0.008, 5e-05, 'Total Profit %= ', 24.006317233830956)
(15, 100, 0.01, 0.006, 7.5e-05, 'Total Profit %= ', 22.946353339963977)
(15, 100, 0.075, 0.006, 7.5e-05, 'Total Profit %= ', 15.48198739134432)
(15, 100, 0.05, 0.006, 7.5e-05, 'Total Profit %= ', 15.48198739134432)
(15, 100, 0.01, 0.007, 7.5e-05, 'Total Profit %= ', 15.76753690594983)
(15, 100, 0.05, 0.007, 7.5e-05, 'Total Profit %= ', 

In [14]:
import csv
f = csv.writer(open("result.csv", "wb+"))

f.writerow(["ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])

for x in csvList:
    f.writerow([x['params'][2],
                x['params'][3],
                x['params'][4],
                x['params'][5],
                x['params'][6],
                x["result"]])

In [15]:
import csv
f = csv.writer(open("result.csv", "wb+"))

f.writerow(["ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])

for x in csvList:
    f.writerow([x['params'][2],
                x['params'][3],
                x['params'][4],
                x['params'][5],
                x['params'][6],
                x["result"]])